In [5]:
import pathlib

PDF_DIR = pathlib.Path("data/raw")
pdf_paths = sorted(PDF_DIR.glob("*.pdf"))
assert pdf_paths, "No PDFs found - check path"

from langchain_community.document_loaders import UnstructuredPDFLoader, PyPDFLoader
from langchain.docstore.document import Document
from tqdm import tqdm

source_docs = []

for path in tqdm(pdf_paths, desc="Reading PDFs"):
    try:
        loader = PyPDFLoader(str(path))
        pages = list(loader.lazy_load())
    except Exception as e:
        print(f"Failed at {path.name}: {e}")
        continue
    for p in pages:
        # Unstructured returns one Document per page by default
        p.metadata["source"] = path.stem                 # short source name
        p.metadata["page_num"] = p.metadata.get("page")  # keep original page index
    source_docs.extend(pages)

Reading PDFs: 100%|██████████| 13/13 [00:10<00:00,  1.22it/s]


In [6]:
import re

for d in source_docs :
    txt = d.page_content
    txt = re.sub(r"\s+\n", "\n", txt)
    txt = re.sub(r"\n{3,}", "\n\n", txt)
    d.page_content = txt.strip()

In [7]:
from transformers import AutoTokenizer
from langchain.text_splitter import RecursiveCharacterTextSplitter

spliiter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    AutoTokenizer.from_pretrained("thenlper/gte-small"),
    chunk_size = 200,
    chunk_overlap=20,
    add_start_index = True,
    strip_whitespace = True
)

print("Splitting and deduplicating...")

docs_processed, seen = [], set()

for doc in tqdm(source_docs) :
    for chunk in spliiter.split_documents([doc]):
        if chunk.page_content not in seen:
            seen.add(chunk.page_content)
            docs_processed.append(chunk)

Splitting and deduplicating...


100%|██████████| 432/432 [00:03<00:00, 137.12it/s]


The above block is used to process raw text content into FAISS's expected format.

In [8]:
## SANITY CHECK - MAKE SURE THAT OUR CODE WORKED AND DOCUMENTS WERE CREATED

doc = docs_processed[15]
print(type(doc))
print(doc.metadata)
print(doc.page_content[:1000])

<class 'langchain_core.documents.base.Document'>
{'producer': 'iLovePDF', 'creator': 'Acrobat PDFMaker 11 for Word', 'creationdate': '2023-12-06T15:01:50+05:30', 'author': 'Amol Dighe', 'company': '', 'sourcemodified': 'D:20231206092752', 'subject': 'A Roadmap prepared by the Indian Nuclear Physics Communitywith TIFR, Mumbai as the Nodal Scientific Institution', 'title': 'Mega Science Vision – 2035   Nuclear Physics', 'moddate': '2024-01-24T10:12:18+00:00', 'source': 'DST - MSV2035-NP-Final', 'total_pages': 140, 'page': 14, 'page_label': '15', 'page_num': 14, 'start_index': 0}
MEGA SCIENCE VISION – 2035   NUCLEAR PHYSICS
3
THE DRAFTING AND WO RKING GROUPS
Director TIFR, Mumbai –
Dr. Jayaram Chengalur / Dr. S. Ramakrishnan / Dr. Sandip Trivedi Chairperson
Members from the D rafting Group
Dr. Alphonsa Joseph Palakkel, IPR, Gandhinagar Member
Dr. Aradhana Srivastava, BARC, Mumbai Member
Dr. Bedangadas Mohanty, NISER, Bhubaneswar Member
Dr. Rudrajyoti Palit, TIFR, Mumbai Member
Other exper

In [9]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.vectorstores.utils import DistanceStrategy

embed = HuggingFaceEmbeddings(model_name="thenlper/gte-small")

vector_db = FAISS.from_documents(docs_processed, embedding=embed, distance_strategy = DistanceStrategy.COSINE)

FAISS.save_local(vector_db, "data/processed/faiss_index")

/var/folders/x8/mz642z_s78s28vthdq18syww0000gn/T/ipykernel_36754/2057286766.py:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embed = HuggingFaceEmbeddings(model_name="thenlper/gte-small")


In [33]:
from rapidfuzz import fuzz, process  # lightweight fuzzy search
from spellchecker import SpellChecker
spell = SpellChecker()

def spellfix(text: str) -> str:
    corrected_words = []
    for word in text.split():
        if word.lower() not in spell:
            corrected = spell.correction(word)
            corrected_words.append(corrected if corrected else word)
        else:
            corrected_words.append(word)
    return " ".join(corrected_words)


class FuzzyRetrieverTool(RetrieverTool):
    def forward(self, query: str):
        query = spellfix(query)     # write your own or use `pyspellchecker`
        return super().forward(query)


In [35]:
from smolagents import Tool
from langchain.vectorstores import VectorStore

class RetrieverTool(Tool):
    name = "retriever"
    description = """
        Using semantic similarity, retrieves some documents from the knowledge base that have the closest embeddings to the input query.
        Always call this tool **before** answering any question that might be answered using documents from the DST / ISRO PDF knowledge base.
        Return the 3-7 most relevant passages.
        """
    
    inputs = {
        "query": {
            "type": "string",
            "description": "The query to perform. This should be semantically close to your target documents. Use the affirmative form rather than a question.",
        }
    }
    output_type = "string"

    def __init__(self, vector_db: VectorStore, **kwargs):
        super().__init__(**kwargs)
        self.vector_db = vector_db

    def forward(self, query: str):
        assert isinstance(query, str), "Your search query must be a string"
        cleaned_query = spellfix(query)
        docs = self.vector_db.similarity_search(
            query=cleaned_query,
            k=7
        )
        return "\nRetrieved documents:\n" + "".join(
            [f"===== Document {str(i)} =====\n" + doc.page_content for i, doc in enumerate(docs)]
        )

In [40]:
from smolagents import LiteLLMModel, ToolCallingAgent
import os

model = LiteLLMModel(model_id="gemini/gemini-2.0-flash-lite", api_key=os.getenv('GEMINI_API_KEY'))

retriever_tool = RetrieverTool(vector_db)
agent = ToolCallingAgent(tools=[retriever_tool], model=model, instructions=(
            """You are an answer-bot specialised in the Indian space-science PDFs I just gave you. "
            Workflow:\n
            1. ALWAYS call the `retriever` tool with the user's query.\n
            2. Read its passages, then craft a concise answer.\n
            3. Call `final_answer` with that answer.\n"""
        ))

In [41]:
output = agent.run("What are himalyan heights most suitable for?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What are himalyan heights most suitable for?                                                                    │
│                                                                                                                 │
╰─ LiteLLMModel - gemini/gemini-2.0-flash-lite ───────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'retriever' with arguments: {'query': 'Himalayan heights suitability'}                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: Retrieved documents:
===== Document 0 =====
MENU
Home  >> Himalayan heights potentially perfect for India's 'Quantum Leap' to space: Study
Himalayan heights potentially perfect for India's 'QuantumLeap' to space: Study
In a pioneering study for the Indian subcontinent, scientists have mapped out optimal locations for
beaming quantum signals into space.
Satellite-based quantum communications including quantum key distribution (QKD) represent one of
the most promising approaches toward global-scale quantum communications. To determine the
viability of transmitting quantum signals through the atmosphere, it is essential to conduct
atmospheric simulations for both uplink and downlink quantum communications and practicality of
potential locations for the same need to be determined.
In meticulous research, Raman Research Institute (RRI) scientists analysed existing open-source data
available on three of India's most sophisticated observatory sites, and found that the Indian
Astronomical Observatory (IAO) in Hanle, nestled in the pristine heights of Ladakh, as the prime
candidate for this revolutionary technology.===== Document 1 =====
earth. Later, through a series of highly complex
manoeuvres, the desired trajectories will be achieved.
After circling the Earth in its T ransfer Orbit,
Chandrayaan-1 spacecraft will be taken into more
elliptical ‘Extended T ransfer Orbits’ by repeatedly firing
its Liquid Apogee Motor (LAM) in a pr-determined
sequence. Subsequently, the LAM is again fired to
make the spacecraft to travel to the vicinity of the
moon.
When it reaches the vicinity of the Moon and passes
at a few hundred kilometers from it, its LAM is fired
again so that the spacecraft slows down sufficiently
to enable the gravity of the moon to capture it into
an elliptical orbit.
Following this, the height of the spacecraft’s orbit
around the moon is reduced in steps. After a careful
and detailed observation of the orbit perturbations
there, the orbital height of Chandrayaan-1 will be
finally lowered to its intended 100 km height from===== Document 2 =====
in elemental composition, and hence can serve as an excellent ground truth for future
remote sensing missions.
While these results have been from the Pragyan rover, the Vikram lander has brought
out some insight about the lunar ground vibrations at the Shiv Shakti point. The ILSA
payload onboard the Vikram lander has recorded over 250 distinct events, of which about
200 records are correlated to a few known activities involving physical movements by
the rover or instruments for science experiments. The results have been published in the
journal Icarus (https://doi.org/10.1016/j.icarus.2024.116285).
Chandrayaan-3 has received the following prestigious awards:
i. Rashtriya Vigyan (Team) Puraskar – Team
ii. International Astronautical Federation (IAF) Award for Excellence in Space
Exploration
iii. John L. ‘Jack’ Swigert Jr. Award for Space Exploration===== Document 3 =====
India and the other with Indian contribution), one
from Bulgaria and two from the United States.
a) Indian payloads
T errain Mapping Camera (TMC), a CCD camera
that maps the topography of the moon, which helps
in better understanding of the lunar evolution process.
Hyperspectral Imager (HySI) , another CCD
camera, is designed for mapping of the minerals on
the lunar surface as well as for understanding the
mineralogical composition of Moon’s interior.
Lunar Laser Ranging Instrument (LLRI) provides
necessary data for accurately determining the height
of lunar surface features.
High Energy X-ray Spectrometer (HEX)  is
designed to help explore the possibility of identifying
Polar Regions covered by thick water-ice deposits as
well as in identifying regions of high Uranium and
Thorium concentrations.
Moon Impact Probe (MIP)  demonstrates the
technologies required for landing a probe at the
desired location on the moon. It is also intended to===== Document 4 =====
candidate for this revolutionary technology.
Wh

[Step 1: Duration 2.21 seconds| Input tokens: 1,227 | Output tokens: 7]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 2 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│ Calling tool: 'final_answer' with arguments: {'answer': 'The Himalayan heights, specifically the Indian         │
│ Astronomical Observatory (IAO) in Hanle, are particularly suitable for beaming quantum signals into space due   │
│ to their pristine environment, low atmospheric water vapor, and oxygen concentrations.'}                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Observations: The Himalayan heights, specifically the Indian Astronomical Observatory (IAO) in Hanle, are 
particularly suitable for beaming quantum signals into space due to their pristine environment, low atmospheric 
water vapor, and oxygen concentrations.

Final answer: The Himalayan heights, specifically the Indian Astronomical Observatory (IAO) in Hanle, are 
particularly suitable for beaming quantum signals into space due to their pristine environment, low atmospheric 
water vapor, and oxygen concentrations.

[Step 2: Duration 1.33 seconds| Input tokens: 4,026 | Output tokens: 52]

In [24]:
def semantic_search(query: str, k: int = 4):
    docs_and_scores = vector_db.similarity_search_with_score(query, k=k)
    return [(d.page_content[:200], score, d.metadata) for d, score in docs_and_scores]

print(semantic_search("What are himalyan heights most suitable for?"))

[("MENU\nHome \xa0>>\xa0Himalayan heights potentially perfect for India's 'Quantum Leap' to space: Study\nHimalayan heights potentially perfect for India's 'QuantumLeap' to space: Study\nIn a pioneering study for", np.float32(0.35979202), {'producer': 'Skia/PDF m133', 'creator': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.0.0.0 Safari/537.36', 'creationdate': '2025-02-25T18:40:23+00:00', 'title': "Himalayan heights potentially perfect for India's 'Quantum Leap' to space: Study | Department Of Science & Technology", 'moddate': '2025-02-25T18:40:23+00:00', 'source': 'Himalayan heights potentially perfect for India_s _Quantum Leap_ to space_ Study _ Department Of Science & Technology', 'total_pages': 3, 'page': 0, 'page_label': '1', 'page_num': 0, 'start_index': 0}), ('India and the other with Indian contribution), one\nfrom Bulgaria and two from the United States.\na) Indian payloads\nT errain Mapping Camera (TMC), a CCD camera\nthat